# BigQuery Sample Codes  
DataFabric 분석환경에서 제공하는 JupyerHub에서 pydatafabric 파이썬 패키지를 사용하면 BigQuery와 연동하여 분석 및 모델링 작업을 수행할 수 있습니다.
pydatafabric 패키지는 분석환경에 기본적으로 설치되어 있으며 pip를 사용하여 버전 업그레이드를 할 수 있습니다.
~~~bash
$ pip install --upgrade pydatafabric
~~~

## BigQuery 실행  
다음과 같이 2가지 방법으로 쿼리를 실행하고 결과를 확인할 수 있습니다.  
- IPython Magic을 사용하여 Jupyter Notebook Cell에서 SQL 실행
- BigQuery Client 사용하여 SQL 실행

### IPython Magic으로 SQL 실행  
Jupyter Notebook Cell에서 SQL을 실행하고 그 결과를 확인할 수 있습니다. 쿼리 결과는 변수에 Pandas Dataframe으로 저장할 수 있습니다. Ditonary로 정의된 쿼리 파라미터를 SQL에 주입할 수 있습니다.

먼저, IPython Magic을 로드합니다.

In [1]:
from pydatafabric.gcp import load_bigquery_ipython_magic

load_bigquery_ipython_magic()

다음과 같이 SQL을 실행합니다.

In [2]:
%%bq
    SELECT MAX(dt) as max_dt
    FROM smart-ruler-304409.eapp_data_dev.test_eapp_data

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.05rows/s]

BigQuery execution took 3 seconds.


,max_dt
0,2022-08-30


SQL 결과를 Pandas DataFrame으로 변수에 저장할 수 있습니다. 다음은 max_dt라는 변수에 SQL 결과를 저장하는 예제입니다.

In [3]:
from pydatafabric.gcp import load_bigquery_ipython_magic

load_bigquery_ipython_magic()

In [4]:
%%bq max_dt

SELECT MAX(dt) as value
FROM smart-ruler-304409.eapp_data_dev.test_eapp_data

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.15rows/s]

BigQuery execution took 2 seconds.


In [10]:
max_dt

,value
0,2022-08-30


쿼리 파라미터를 SQL에 전달할 수 있습니다. 전달할 값은 Dictionary 타입으로 정의합니다.

In [21]:
import json

query_params = {
    "max_dt": max_dt["value"][0].strftime("%Y-%m-%d")
}
query_params

{'max_dt': '2022-08-30'}

다음과 같이 params 옵션에 전달할 쿼리 파라미터 변수를 정의합니다.

In [23]:
%%bq --params $query_params

SELECT *
FROM smart-ruler-304409.eapp_data_dev.test_eapp_data
WHERE dt = '{max_dt}'

### BigQuery Client로 SQL 실행  
BigQuery Client는 Google에서 제공하는 Client Library에서 제공하는 객체입니다. SQL을 실행할 수 있을 뿐만 아니라 Google Client Library에서 제공하는 다양한 기능들을 활용할 수 있습니다. 더욱 자세한 내용은 <a href="https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client" target="_blank">이곳</a>에서 확인하시기 바랍니다.

다음과 같이 BigQuery Client를 생성하여 SQL을 실행합니다. 쿼리 조회 결과가 있는 경우 Iterator 형태로 결과를 리턴해줍니다.

In [24]:
from pydatafabric.gcp import get_bigquery_client

bq_client = get_bigquery_client()

sql = """
    SELECT *
    FROM smart-ruler-304409.eapp_data_dev.test_eapp_data
    WHERE dt = '{max_dt}'
    LIMIT 5
""".format(**query_params)

iterator = bq_client.query(sql).result()
for r in iterator:
    print(r)

Row(('2208301038576463', '63b5d5b38949e396baef709bc9ed34c7ea997d44a6fa36aff8c0e9709dc9687b', '2', '여', '66', '60', '1026', '이마트 가양점', 10, '고추가격은그런대로~~~~~', 14, '2500000195051', '하루채소 오이맛고추', datetime.date(2022, 8, 29), datetime.date(2022, 8, 30), '13', '채소', '10', '신선1담당', '135', '과채소', '9684', '고추류', '0105', '오이맛고추', '00', '서울', Decimal('126.861440467'), Decimal('37.557922427'), 10, 0, 0, '001', 'N', 'Y', '135::1::6,135::2::6,135::3::6,135::4::6,135::5::6', Decimal('6')), {'review_id': 0, 'cust_id': 1, 'gender_cd': 2, 'gender_nm': 3, 'age': 4, 'agrde_cd_10_unit': 5, 'store_cd': 6, 'store_nm': 7, 'score': 8, 'comments': 9, 'comments_length': 10, 'sku_cd': 11, 'sku_nm': 12, 'order_dt': 13, 'dt': 14, 'prdt_cat_cd': 15, 'prdt_cat_nm': 16, 'prdt_di_cd': 17, 'prdt_di_nm': 18, 'prdt_gcode_cd': 19, 'prdt_gcode_nm': 20, 'prdt_mcode_cd': 21, 'prdt_mcode_nm': 22, 'prdt_dcode_cd': 23, 'prdt_dcode_nm': 24, 'area_cd': 25, 'area_nm': 26, 'longitude': 27, 'latitude': 28, 'comments_point': 29, 'image_

### INSERT OVERWRITE
BigQuery는 Hive와 달리 INSERT OVERWRITE 기능을 제공하지 않습니다. 그러나 emart 패키지의 bq_insert_overwrite 메서드를 사용하면 INSERT OVERWRITE가 가능합니다.  
만약 테이블이 존재하지 않는다면 새로 테이블을 생성합니다.  
저장하려는 대상 테이블이 파티셔닝되어 있다면 partition 파라미터를 사용하여 파티션 컬럼 이름을 설정합니다. sql 결과의 파티션 컬럼 값에 따라 해당 파티션으로 저장됩니다.  
대상 테이블이 cluster 설정이 되어 있거나 새로 생성하는 테이블에 clustrer를 설정하고 싶은 경우 clustering_fields 파라미터에 컬럼 이름을 리스트로 넣어줍니다.

In [25]:
from pydatafabric.gcp import bq_insert_overwrite, get_temp_table

sql = """
    SELECT *
    FROM smart-ruler-304409.eapp_data_dev.test_eapp_data
    WHERE dt = '{max_dt}'
""".format(**query_params)

result_table_name= get_temp_table()

bq_insert_overwrite(sql=sql, destination=result_table_name, partition="dt", clustering_fields=["cust_id"])

destination: emart-datafabric.temp_1d.ccb2ad72_fe96_470a_a363_c66941b47f38
total_rows: 25608
slot_secs: 4.093



### Multi Partitions
BigQuery는 단일 컬럼 파티션만 제공합니다. BigQuery에서 멀티 파티션 컬럼을 사용할 수는 없지만 BigQuery에서 제공하는 와일드카드 테이블을 응용하여 멀티 파티션 테이블처럼 정의할 수 있습니다.  
<b>"__"</b>를 구분자로 한 테이블들을 생성하여 멀티 파티션처럼 사용하는 예제입니다.

In [26]:
%%bq --params $query_params

CREATE OR REPLACE TABLE temp_1d.wildcard_table__subpart_1
AS
SELECT *
FROM {result_table_name}
WHERE dt = '{max_dt}'
;

CREATE OR REPLACE TABLE temp_1d.wildcard_table__subpart_2
AS
SELECT *
FROM {result_table_name}
WHERE dt = '{max_dt}'
;

SELECT *
FROM `temp_1d.wildcard_table__*`
WHERE _TABLE_SUFFIX = 'subpart_1'
LIMIT 5
;

SyntaxError: --params is not a correctly formatted JSON string or a JSON serializable dictionary (<string>)

이렇게 와일드카드로 생성한 테이블들에도 INSERT OVERWRITE를 사용할 수 있습니다. emart 패키지의 bq_insert_overwrite_with_suffixes 메서드를 사용하여 특정 suffix를 가지는 테이블의 특정 파티션에 INSERT OVERWRITE 할 수 있습니다. 마치 멀티 파티션 테이블에 데이터를 저장하는 것처럼 말입니다.  
아래와 같이 suffixes 파라미터에 추가 파티션으로 사용할 컬럼 이름을 지정합니다. suffix가 여러 개인 경우(예: table_name__subpart1__subpart2)도 suffixes 파라미터에 해당 컬럼 이름들을 리스트로 넣어주면 INSERT OVERWRITE 할 수 있습니다.

In [ ]:
from pydatafabric.gcp import bq_insert_overwrite, get_temp_table

sql = f"""
    SELECT *, 'subpart_1' as subpart
    FROM smart-ruler-304409.eapp_data_dev.test_eapp_data
    WHERE dt = '{max_dt}'
""".format(**query_params)

result_table_name= get_temp_table()

bq_insert_overwrite(sql=sql, destination=result_table_name, suffixes=["subpart"], partition="dt", clustering_fields=["product_grp_id"])

## BigQuery to Pandas
위에서 설명했듯이 BigQuery 쿼리 결과를 Pandas Dataframe으로 리턴받을 수 있습니다.
위의 IPython Magic 뿐만 아니라 emart 패키지에서 제공하는 메서드를 사용할 수도 있습니다.   
   
(참고 : Jupyter Notebook Cell에서 IPyhon Magic을 사용하여 SQL을 실행하는 경우 그 Cell에는 다른 Python 코드를 추가할 수 없습니다. Cell에 다른 Python 코드를 자유롭게 추가하려면 emart 패키지의 기능을 활용하시기 바랍니다.)


다음은 bq_to_pandas 메서드로 결과를 Pandas Dataframe에 저장하는 예입니다.

In [28]:
from pydatafabric.gcp import bq_to_pandas

pd_df = bq_to_pandas(f"""
    SELECT *
    FROM {result_table_name}
    WHERE dt = (SELECT MAX(dt) FROM {result_table_name})
""")

pd_df

destination: emart-datafabric._2dd36219768c7c869a5680edf9fd6e104ea57800.anon9d695840ddac56383bfa74b8e17a4cbde45d65fe3634746b1bab5873b729709d
total_rows: 25608
slot_secs: 1.256



Downloading: 100%|██████████| 25608/25608 [00:00<00:00, 28413.16rows/s]


,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208300717592839,ffd6bf62621d2dcd1944db24adfa5a02b1fbf57add2226...,2,여,51,50,1120,이마트 미아점,10,"자주먹고, 매일 마시는데, 가격이 조금만 낮아져도 좋을 듯합니다,",...,127.030022270,37.610847692,10,10,0,003,N,Y,"332::1::10,332::2::10,332::3::8,332::4::10,332...",10.000000000
1,2208300828272912,7ea94cb7154eac653b5a1eda89907270b65cbc4c6c2b9b...,2,여,54,50,1000,이마트 창동점,10,ㄴㄱㅅㅇㅈ뒥ㅂㅅㅇㅈㅅ노,...,127.046690683,37.651608368,10,0,0,001,Y,Y,"342::1::10,342::2::10,342::3::10,342::4::10,34...",10.000000000
2,2208300707556891,86d736498822620ee8ad5a6228627caebccc73fbcfd801...,2,여,48,40,1000,이마트 창동점,10,아이가좋아해요~~~,...,127.046690683,37.651608368,10,0,0,001,N,Y,"340::1::10,340::2::10,340::3::10,340::4::10,34...",10.000000000
3,2208300006452516,239ec5061e62ff2d3411da43367d84f0f4a8e29b90f06c...,2,여,87,70,1000,이마트 창동점,10,오픈런까지 한 보람은 좀 ..,...,127.046690683,37.651608368,10,10,0,003,N,Y,"154::1::10,154::2::10,154::3::10,154::4::8,154...",10.000000000
4,2208300008031494,b5255a4eb0e00995a1ab4c560b91913cb487cdd2cb64fc...,2,여,56,50,1000,이마트 창동점,10,가격 대비 만족스러웠음~,...,127.046690683,37.651608368,10,0,0,001,N,Y,"154::1::10,154::2::10,154::3::10,154::4::10,15...",10.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2208301158538227,553497183b155892de10e30c28381f06406ce27f457341...,2,여,42,40,7407,토이킹덤 안성점,10,,...,127.147249595,36.994633421,0,0,0,000,N,Y,"542::1::10,542::2::10,542::3::10,542::4::10",10.000000000
25604,2208300440306896,ec8c3f6f08da9871739cd9273b886b001994de33b5167c...,1,남,48,40,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000
25605,2208301522157501,14611f676a02617050fd2484f8ade5c8b9292111ea5e5e...,1,남,51,50,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"534::1::10,534::2::10,534::3::6,534::4::8",9.000000000
25606,2208300915461856,b4090107c89a0eb834d47e9a35fb80b74993db103bf1c0...,2,여,33,30,7714,일렉트로마트 스타필드 고양점,10,디자인 이쁘고 좋아요,...,126.894776182,37.646978959,10,0,0,001,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000


## Pandas to BigQuery

Pandas Dataframe을 특정 BigQuery 테이블에 저장할 수 있습니다.  
다음과 같이 pandas_to_bq_table 메서드를 사용합니다.

In [29]:
import time
from pydatafabric.gcp import pandas_to_bq_table

dest_table = f"eapp_review_data_{str(int(time.time()))}"
print(f"저장할 테이블 : temp_1d.{dest_table}")

pandas_to_bq_table(pd_df, "temp_1d", dest_table)

get_bigquery_client().query(f"""
    SELECT *
    FROM temp_1d.{dest_table}
""").result()

저장할 테이블 : temp_1d.eapp_review_data_1664523449


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/30 16:37:32 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/09/30 16:37:32 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/09/30 16:37:32 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/09/30 16:37:33 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
22/09/30 16:37:43 WARN org.apache.spark.sql.catalyst.util.package: Truncated the stri

데이터가 테이블에 저장되었는지 확인합니다.

In [30]:
query_params =  {"dest_table": dest_table}

In [31]:
%%bq --params $query_params

SELECT *
FROM temp_1d.{dest_table}

SyntaxError: --params is not a correctly formatted JSON string or a JSON serializable dictionary (<string>)

## BigQuery to Spark  

BigQuery 데이터를 가져와서 Spark로 처리할 수 있습니다. BigQuery SQL 결과를 Spark Dataframe으로 변환한 후 이어서 데이터 처리가 가능합니다.

In [35]:
from pydatafabric.gcp import bq_to_df

spark_df = bq_to_df(f"""
    SELECT sku_cd
          ,sku_nm
          ,dt
    FROM temp_1d.{dest_table}
""")

spark_df.head(5)

[Row(sku_cd='2500000039232', sku_nm='근대', dt=datetime.date(2022, 8, 30)),
 Row(sku_cd='2500000039232', sku_nm='근대', dt=datetime.date(2022, 8, 30)),
 Row(sku_cd='2500000039232', sku_nm='근대', dt=datetime.date(2022, 8, 30)),
 Row(sku_cd='2500000039232', sku_nm='근대', dt=datetime.date(2022, 8, 30)),
 Row(sku_cd='2500000039232', sku_nm='근대', dt=datetime.date(2022, 8, 30))]

## Spark to BigQuery  

Spark으로 처리한 결과를 다시 BigQuery에 적재가 가능합니다.  
다음은 위에서 spark_df 변수에 저장한 Spark Dataframe을 다시 BigQuery에 적재하는 예제입니다.

1. 먼저 저장할 테이블을 BigQuery에 생성합니다. 여기서는 파티션된 테이블을 생성하고 특정 파티션에 Spark Dataframe을 저장할 것입니다.

In [36]:
from pydatafabric.gcp import df_to_bq_table

dest_dataset = "temp_1d"
partitioned_dest_table = f"eapp_review_data_{str(int(time.time()))}"

get_bigquery_client().query(f"""
    CREATE OR REPLACE TABLE {dest_dataset}.{partitioned_dest_table}
    (
        sku_cd STRING,
        sku_nm STRING,
        dt DATE
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : temp_1d.eapp_review_data_1664523860


2. Spark Dataframe을 BigQuery 테이블에 저장합니다. 여기서는 파티션 컬럼 타입이 Date이므로 타입 변환 작업을 해주었습니다. 이렇게 경우에 따라 타입 변환 작업이 필요한 경우가 있으므로 타입에 주의해주세요.

In [37]:
changed_df = spark_df.select("sku_cd", "sku_nm", spark_df.dt.cast("date"))
partition_dt = changed_df.head(1)[0].dt.strftime("%Y%m%d")
df_to_bq_table(df=changed_df,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               partition=partition_dt,
               mode="overwrite")

3. BigQuery에 저장되었는지 확인합니다.

In [38]:
query_params = {"dataset":dest_dataset, "table_name":partitioned_dest_table}

In [39]:
%%bq --params $query_params
    SELECT *
    FROM {dataset}.{table_name}

SyntaxError: --params is not a correctly formatted JSON string or a JSON serializable dictionary (<string>)